# Specifying Set Arithmetic for Joins

- The set arithmetic comes up when a value appears in one key column but not the other


- By dafault, the result contains the intersection of the two sets of inputs; this is known as **inner** join


- An **outer** join returns a join over the union of the input columns, and fills in all missing values with NAs.


- The **left join** and the **right join** return join over the left entries and right entries, respectively.


In [1]:
import numpy as np
import pandas as pd

In [3]:
df1 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'bread', 'beans']},
                     columns = ['name', 'food'])

df2 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['soda', 'wine']},
                     columns = ['name', 'drink'])


print(df1)
print()
print(df2)
print()
print(pd.merge(df1, df2)) #inner-intersection(default)

    name   food
0  Peter   fish
1   Paul  bread
2   Mary  beans

     name drink
0    Mary  soda
1  Joseph  wine

   name   food drink
0  Mary  beans  soda


In [4]:
df1 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'bread', 'beans']},
                     columns = ['name', 'food'])

df2 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['soda', 'wine']},
                     columns = ['name', 'drink'])


print(df1)
print()
print(df2)
print()
print(pd.merge(df1, df2, how = 'outer'))  #union

    name   food
0  Peter   fish
1   Paul  bread
2   Mary  beans

     name drink
0    Mary  soda
1  Joseph  wine

     name   food drink
0   Peter   fish   NaN
1    Paul  bread   NaN
2    Mary  beans  soda
3  Joseph    NaN  wine


In [5]:
df1 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'bread', 'beans']},
                     columns = ['name', 'food'])

df2 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['soda', 'wine']},
                     columns = ['name', 'drink'])


print(df1)
print()
print(df2)
print()
print(pd.merge(df1, df2, how = 'left'))

    name   food
0  Peter   fish
1   Paul  bread
2   Mary  beans

     name drink
0    Mary  soda
1  Joseph  wine

    name   food drink
0  Peter   fish   NaN
1   Paul  bread   NaN
2   Mary  beans  soda


In [6]:
df1 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'bread', 'beans']},
                     columns = ['name', 'food'])

df2 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['soda', 'wine']},
                     columns = ['name', 'drink'])


print(df1)
print()
print(df2)
print()
print(pd.merge(df1, df2, how = 'right'))

    name   food
0  Peter   fish
1   Paul  bread
2   Mary  beans

     name drink
0    Mary  soda
1  Joseph  wine

     name   food drink
0    Mary  beans  soda
1  Joseph    NaN  wine


In [9]:
## Overlapping Column names:


df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1,2,3,4]},
                     columns = ['name', 'rank'])

df4 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3,1,4,2]},
                     columns = ['name', 'rank'])


print(df3)
print()
print(df4)
print()
print(pd.merge(df3, df4, on = 'name'))

   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2


In [10]:
print(pd.merge(df3, df4, on = 'name', suffixes = ['_L', '_R']))

   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2


## Example:  US States Data

In [15]:
pop = pd.read_csv('population.csv')
areas = pd.read_csv('areas.csv')
abbrevs = pd.read_csv('abbrevs.csv')

print(pop.head())
print()
print(areas.head())
print()
print(abbrevs.head())

  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0

        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA


In [17]:
print(pop.shape)
print(areas.shape)
print(abbrevs.shape)

(2544, 4)
(52, 2)
(51, 2)


In [19]:
pop['state/region'].unique().shape

(53,)

In [20]:
abbrevs['abbreviation'].unique().shape

(51,)

In [21]:
merged = pd.merge(pop, abbrevs, how = 'outer', left_on = 'state/region', right_on = 'abbreviation')

merged.head()

,state/region,ages,year,population,state,abbreviation
0,AL,under18,2012,1117489.0,Alabama,AL
1,AL,total,2012,4817528.0,Alabama,AL
2,AL,under18,2010,1130966.0,Alabama,AL
3,AL,total,2010,4785570.0,Alabama,AL
4,AL,under18,2011,1125763.0,Alabama,AL


In [22]:
merged.drop('abbreviation', axis = 'columns', inplace = True)
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


In [26]:
merged.isnull().any() #sum()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [30]:
merged[merged['population'].isnull()]

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN
2453,PR,under18,1993,NaN,NaN
2454,PR,under18,1992,NaN,NaN
2455,PR,total,1992,NaN,NaN
2456,PR,under18,1994,NaN,NaN
2457,PR,total,1994,NaN,NaN


In [34]:
merged[merged['state/region'] == 'PR']

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN
2453,PR,under18,1993,NaN,NaN
2454,PR,under18,1992,NaN,NaN
2455,PR,total,1992,NaN,NaN
2456,PR,under18,1994,NaN,NaN
2457,PR,total,1994,NaN,NaN


- The null values in population column are from Puerto Rico prior to the year 2000


In [38]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

In [41]:
merged

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama
...,...,...,...,...,...
2539,USA,total,2010,309326295.0,United States
2540,USA,under18,2011,73902222.0,United States
2541,USA,total,2011,311582564.0,United States
2542,USA,under18,2012,73708179.0,United States


In [39]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'

In [40]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [42]:
merged

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama
...,...,...,...,...,...
2539,USA,total,2010,309326295.0,United States
2540,USA,under18,2011,73902222.0,United States
2541,USA,total,2011,311582564.0,United States
2542,USA,under18,2012,73708179.0,United States


In [46]:
areas.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [47]:
merged['state'].unique().shape

(53,)

In [49]:
areas['state'].unique().shape

(52,)

In [51]:
final = pd.merge(merged, areas, on = 'state', how = 'left')

final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [61]:
final.isnull().any()

state/region     False
ages             False
year             False
population       False
state            False
area (sq. mi)    False
dtype: bool

In [57]:
final.loc[final['area (sq. mi)'].isnull(), 'state'].unique()

array(['United States'], dtype=object)

In [58]:
final.shape

(2544, 6)

In [60]:
final.dropna(inplace = True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [68]:
data2010 = final[(final['year'] == 2010) & (final['ages'] == 'total')]

data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


In [72]:
data2010.reset_index(drop = True).head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,total,2010,4785570.0,Alabama,52423.0
1,AK,total,2010,713868.0,Alaska,656425.0
2,AZ,total,2010,6408790.0,Arizona,114006.0
3,AR,total,2010,2922280.0,Arkansas,53182.0
4,CA,total,2010,37333601.0,California,163707.0


In [74]:
data2010.set_index('state', inplace = True)

KeyError: "None of ['state'] are in the columns"

In [75]:
data2010.head()

,state/region,ages,year,population,area (sq. mi)
state,,,,,
Alabama,AL,total,2010,4785570.0,52423.0
Alaska,AK,total,2010,713868.0,656425.0
Arizona,AZ,total,2010,6408790.0,114006.0
Arkansas,AR,total,2010,2922280.0,53182.0
California,CA,total,2010,37333601.0,163707.0


In [82]:
density= data2010['population'] / data2010['area (sq. mi)']

In [83]:
density.head()

state
Alabama        91.287603
Alaska          1.087509
Arizona        56.214497
Arkansas       54.948667
California    228.051342
dtype: float64

In [85]:
density.sort_values(ascending= False, inplace = True)

density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

In [86]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

## Aggregation and Grouping

- An essential part of analysis of large data is efficient summarization

- Computations like sum, mean, median, min, max, etc. in which a single number gives insight into the nature of a     large dataset

In [87]:
import seaborn as sns

In [128]:
planets = sns.load_dataset('planets')
planets.shape

(1035, 6)

In [129]:
planets

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300000,7.10,77.40,2006
1,Radial Velocity,1,874.774000,2.21,56.95,2008
2,Radial Velocity,1,763.000000,2.60,19.84,2011
3,Radial Velocity,1,326.030000,19.40,110.62,2007
4,Radial Velocity,1,516.220000,10.50,119.47,2009
...,...,...,...,...,...,...
1030,Transit,1,3.941507,NaN,172.00,2006
1031,Transit,1,2.615864,NaN,148.00,2007
1032,Transit,1,3.191524,NaN,174.00,2007
1033,Transit,1,4.125083,NaN,293.00,2008


In [130]:
planets['method'].unique()

array(['Radial Velocity', 'Imaging', 'Eclipse Timing Variations',
       'Transit', 'Astrometry', 'Transit Timing Variations',
       'Orbital Brightness Modulation', 'Microlensing', 'Pulsar Timing',
       'Pulsation Timing Variations'], dtype=object)

In [97]:
ser = pd.Series(np.random.rand(5))
ser

0    0.067344
1    0.710199
2    0.826184
3    0.267710
4    0.677546
dtype: float64

In [98]:
ser.sum()

2.5489837638387356

In [99]:
ser.mean()

0.5097967527677472

In [100]:
df = pd.DataFrame({'A': np.random.rand(5),
             ' B': np.random.rand(5)})

df

,A,B
0,0.868975,0.591562
1,0.257306,0.683278
2,0.600845,0.514543
3,0.125612,0.395212
4,0.808357,0.343681


In [101]:
df.mean()

A     0.532219
 B    0.505655
dtype: float64

In [102]:
df.mean(axis = 'columns')

0    0.730269
1    0.470292
2    0.557694
3    0.260412
4    0.576019
dtype: float64

In [107]:
planets.isnull().any()

method            False
number            False
orbital_period    False
mass              False
distance          False
year              False
dtype: bool

In [131]:
planets.dropna()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.30000,7.100,77.40,2006
1,Radial Velocity,1,874.77400,2.210,56.95,2008
2,Radial Velocity,1,763.00000,2.600,19.84,2011
3,Radial Velocity,1,326.03000,19.400,110.62,2007
4,Radial Velocity,1,516.22000,10.500,119.47,2009
...,...,...,...,...,...,...
640,Radial Velocity,1,111.70000,2.100,14.90,2009
641,Radial Velocity,1,5.05050,1.068,44.46,2013
642,Radial Velocity,1,311.28800,1.940,17.24,1999
649,Transit,1,2.70339,1.470,178.00,2013


In [ ]:
#mean(), median()
#min(), max()
#std(), var()
#sum()
#count()

In [108]:
planets.describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


1, 11, 15, 19, 20, 24, 28, 34, 37 , 47, 50 ,57

1st quartile(25%)

In [109]:
## GroupBy: Split, Apply, Combine

In [111]:
df = pd.DataFrame({"key": ['A','B', 'C', 'A', 'B', 'C'],
                   'data': range(6)})

df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [115]:
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


In [132]:
planets.groupby('method')

In [133]:
#Column indexing
planets.groupby('method')['orbital_period']

In [134]:
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

In [136]:
planets.groupby('method')['year'].describe()

,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
Astrometry,2.0,2011.500000,2.121320,2010.0,2010.75,2011.5,2012.25,2013.0
Eclipse Timing Variations,9.0,2010.000000,1.414214,2008.0,2009.00,2010.0,2011.00,2012.0
Imaging,38.0,2009.131579,2.781901,2004.0,2008.00,2009.0,2011.00,2013.0
Microlensing,23.0,2009.782609,2.859697,2004.0,2008.00,2010.0,2012.00,2013.0
Orbital Brightness Modulation,3.0,2011.666667,1.154701,2011.0,2011.00,2011.0,2012.00,2013.0
Pulsar Timing,5.0,1998.400000,8.384510,1992.0,1992.00,1994.0,2003.00,2011.0
Pulsation Timing Variations,1.0,2007.000000,NaN,2007.0,2007.00,2007.0,2007.00,2007.0
Radial Velocity,553.0,2007.518987,4.249052,1989.0,2005.00,2009.0,2011.00,2014.0
Transit,397.0,2011.236776,2.077867,2002.0,2010.00,2012.0,2013.00,2014.0


## Vectorized String Operations:

In [138]:
#Pandas String Operations

x = np.array([2,3,4,5,6,11])

x * 2

array([ 4,  6,  8, 10, 12, 22])

In [140]:
data = np.array(['peter', 'Paul', 'MARY'])

# str_data = []
# for string in data:
#     strdata.append(string.capitalize)
    
[s.capitalize() for s in data]

['Peter', 'Paul', 'Mary']

In [141]:
names = pd.Series(data)
names

0    peter
1     Paul
2     MARY
dtype: object

In [142]:
names.str.capitalize()

0    Peter
1     Paul
2     Mary
dtype: object

In [143]:
names = pd.Series(['Max Chapman', 'Terry Jones'])

In [144]:
names.str.lower()

0    max chapman
1    terry jones
dtype: object

In [145]:
names.str.len()

0    11
1    11
dtype: int64

In [146]:
names.str.startswith('T')

0    False
1     True
dtype: bool

In [147]:
names.str.split()

0    [Max, Chapman]
1    [Terry, Jones]
dtype: object